In [1]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [2]:
lieferzeile = pd.read_csv(r'Einkaufslieferzeile.csv', on_bad_lines='skip', sep=';', low_memory=False,encoding='utf-8-sig')
lieferzeile.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,Use Duplication List,Use Tax,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K00421,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,000000005295C63A
1,0,0,NaN,0,NaN,0,NaN,NaN,K02028,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052EC85AD
2,0,0,NaN,0,NaN,0,NaN,NaN,K04610,0,...,0,0,"0,00000000000000000000",EU,1,MWST19,"0,00000000000000000000",NaN,NaN,0000000052B4AE3C
3,0,0,NaN,0,NaN,0,NaN,NaN,K04607,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4D12C
4,0,0,NaN,0,NaN,10000,RB19/0025,NaN,K04137,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4B8F5


In [3]:
lieferzeile.shape

(6012, 166)

In [4]:
# drop columns where all NaN values
lieferzeile=lieferzeile.dropna(axis=1, how='all')
lieferzeile.shape

(6012, 129)

In [5]:
# drop columns where all 0 values
lieferzeile=lieferzeile.loc[:, (lieferzeile != 0).any(axis=0)]
lieferzeile.shape

(6012, 103)

In [6]:
# drop columns where over 50% of values are null values
lieferzeile=lieferzeile.loc[:, lieferzeile.isnull().mean()<0.5]
lieferzeile.shape

(6012, 90)

In [7]:
# get list with column names
list(lieferzeile.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Item Rcpt_ Entry No_',
 'Line No_',
 'No_',
 'Order Line No_',
 'Order No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'Allow Invoice Disc_',
 'Balance',
 'Bin Code',
 'Buy-from Address',
 'Buy-from City',
 'Buy-from Contact',
 'Buy-from Post Code',
 'Buy-from Vendor Name 2',
 'Buy-from Vendor Name',
 'Contract Amount',
 'Correction',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Item Category Code',
 'Item Charge Base Amount',
 'Job Currency Factor',
 'Job Line Amount (LCY)',
 'Job Line Amount',
 'Job Line Disc_ Amount (LCY)',
 'Job Line Discount Amount',
 'Job Line

In [8]:
# drop columns that don't seem relevant to our challenge
to_drop=['Blanket Order Line No_', 'Buy-from Vendor No_', 'Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Item Rcpt_ Entry No_',
 'Line No_', 'Order Line No_', 'Pay-to Vendor No_', 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'Allow Invoice Disc_', 'Balance',
 'Bin Code', 'Contract Amount', 'Correction', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Item Charge Base Amount',
 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Discount _', 'Net Weight', 'Overhead Rate', 'Pack Sample Quantity', 'Price Factor', 
 'Qty_ per Unit of Measure', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code',
 'Type', 'Unit Cost (LCY)', 'Unit Cost', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code',
 'Unit of Measure', 'Units per Parcel', 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Prod_ Posting Group',
 'VAT _', 'Buy-from Vendor Name 2','Buy-from Contact','Buy-from Address', 'timestamp','Item Category Code','Posting Group','Description','Job Total Price']
lieferzeile.drop(to_drop, inplace=True, axis=1)

In [9]:
lieferzeile.head()

,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Location Code,...,Planned Receipt Date,Posting Date,Promised Receipt Date,Qty_ Invoiced (Base),Qty_ Rcd_ Not Invoiced,Quantity (Base),Quantity Invoiced,Quantity,Really Receipt DateTime,Requested Receipt Date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,B-01,...,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"22730,00000000000000000000","0,00000000000000000000","22730,00000000000000000000","22730,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,B-01,...,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"1500,00000000000000000000","0,00000000000000000000","1500,00000000000000000000","15,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,B-01,...,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,"2000,00000000000000000000","0,00000000000000000000","2000,00000000000000000000","2000,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00
3,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,B-01,...,06.01.2020 00:00:00,30.12.2019 00:00:00,06.01.2020 00:00:00,"0,00000000000000000000","0,00000000000000000000","0,00000000000000000000","300,00000000000000000000","300,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00
4,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,8,B-01,...,18.12.2019 00:00:00,30.12.2019 00:00:00,18.12.2019 00:00:00,"8704,00000000000000000000","0,00000000000000000000","8704,00000000000000000000","8704,00000000000000000000","8704,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00


In [10]:
dates=lieferzeile[[
 'Expected Receipt Date', 'Expected Receipt Time', 'FA Posting Date', 'Lead Time Calculation',  'Planned Receipt Date',
 'Order Date', 'Original Date', 'Posting Date', 'Promised Receipt Date', 'Really Receipt DateTime', 'Requested Receipt Date']]
dates.head(3)

,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Planned Receipt Date,Order Date,Original Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,19.12.2019 00:00:00,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00


In [11]:
# **Date columns to keep**
# Order Date: date the order was created. 
# Posting Date: date you want as the Posting Date on the Ledger Entry tables when the order is Shipped and Invoiced.
# Promised Receipt Date: date your vendor promised they would deliver the goods. NAV does not change this date, so it allows you to set a marker to measure your vendor’s performance against what was promised.
# Requested Receipt Date: date you wanted to receive the goods from the vendor. This date is not recalculated in NAV, so once it has been established, it gives you a marker to measure your vendor’s performance against the requested date.
# Really Receipt DateTime

In [12]:
# searching for how Microsoft Nav calculates dates, we'll drop some columns indicating dates, taking into consideration the following:
# requested receipt date - lead time calculation = order date
# requested receipt date + inbound whse. handling time + safety lead time = expected receipt date
# lead time calculation: amount of time the product needs in order to be delivered to your company that the Purchasing Agent needs to be alerted
# Expected Receipt Date: date you currently expect your vendor to deliver the order
# Planned Receipt Date: date the order is expected to arrive at the warehouse location, uses the Requested Receipt Date
to_drop=[ 'Expected Receipt Date', 'Expected Receipt Time','FA Posting Date','Lead Time Calculation','Planned Receipt Date']
lieferzeile.drop(to_drop, inplace=True, axis=1)
lieferzeile.head()


,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Location Code,Order Date,Order Quantity,Original Date,Posting Date,Promised Receipt Date,Qty_ Invoiced (Base),Qty_ Rcd_ Not Invoiced,Quantity (Base),Quantity Invoiced,Quantity,Really Receipt DateTime,Requested Receipt Date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,23.09.2019 00:00:00,"0,00000000000000000000",17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"22730,00000000000000000000","0,00000000000000000000","22730,00000000000000000000","22730,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,15.11.2019 00:00:00,"0,00000000000000000000",17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"1500,00000000000000000000","0,00000000000000000000","1500,00000000000000000000","15,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,19.12.2019 00:00:00,"0,00000000000000000000",19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,"2000,00000000000000000000","0,00000000000000000000","2000,00000000000000000000","2000,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00
3,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,B-01,03.12.2019 00:00:00,"0,00000000000000000000",03.12.2019 00:00:00,30.12.2019 00:00:00,06.01.2020 00:00:00,"0,00000000000000000000","0,00000000000000000000","0,00000000000000000000","300,00000000000000000000","300,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00
4,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,B-01,03.07.2019 00:00:00,"0,00000000000000000000",18.12.2019 00:00:00,30.12.2019 00:00:00,18.12.2019 00:00:00,"8704,00000000000000000000","0,00000000000000000000","8704,00000000000000000000","8704,00000000000000000000","8704,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00


In [13]:
# Qty. Rcd. Not Invoiced Field: specifies how many units, such as pieces, of the ordered item have been received but not yet invoiced.
# Quantity (Base): Inventory is stored in the base unit of measure, and every time that you sell or purchase or use any other unit of measure, NAV needs to know the quantity expressed in base unit of measure, so it can update inventory correctly
# Order Quantity: covers all gross requirements (forecast, sales orders, and also replenishment of the inventory level).

In [14]:
# drop more columns that don't seem relevant to our challenge
to_drop=[ 'Order Quantity','Qty_ Invoiced (Base)',  'Qty_ Rcd_ Not Invoiced',  'Quantity (Base)']
lieferzeile.drop(to_drop, inplace=True, axis=1)

In [15]:
lieferzeile.head(3)

,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Location Code,Order Date,Original Date,Posting Date,Promised Receipt Date,Quantity Invoiced,Quantity,Really Receipt DateTime,Requested Receipt Date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"22730,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,"15,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,"2000,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00


In [16]:
lieferzeile.columns = lieferzeile.columns.str.replace(" ", "_").str.lower()
lieferzeile=lieferzeile.rename(columns={"no_": "lieferzeile_no","order_no_": "order_no","buy-from_city": "buy_from_city",
"buy-from_post_code":"buy_from_post_code", "buy-from_vendor_name":"buy_from_vendor_name"})
lieferzeile.buy_from_vendor_name=lieferzeile.buy_from_vendor_name.str.replace(",", " ")
list(lieferzeile.columns)

['lieferzeile_no',
 'order_no',
 'buy_from_city',
 'buy_from_post_code',
 'buy_from_vendor_name',
 'location_code',
 'order_date',
 'original_date',
 'posting_date',
 'promised_receipt_date',
 'quantity_invoiced',
 'quantity',
 'really_receipt_datetime',
 'requested_receipt_date']

In [17]:
lieferzeile.quantity_invoiced = lieferzeile.quantity_invoiced.map(lambda x: str(x)[:-21])
lieferzeile.quantity = lieferzeile.quantity.map(lambda x: str(x)[:-21])
lieferzeile.head(3)

,lieferzeile_no,order_no,buy_from_city,buy_from_post_code,buy_from_vendor_name,location_code,order_date,original_date,posting_date,promised_receipt_date,quantity_invoiced,quantity,really_receipt_datetime,requested_receipt_date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,22730,22730,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,15,15,01.01.1753 00:00:00,17.12.2019 00:00:00
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,2000,2000,01.01.1753 00:00:00,19.12.2019 00:00:00


In [18]:
lieferzeile = lieferzeile.astype({'quantity_invoiced': 'int64','quantity': 'int64',})
lieferzeile.dtypes

lieferzeile_no             object
order_no                   object
buy_from_city              object
buy_from_post_code         object
buy_from_vendor_name       object
location_code              object
order_date                 object
original_date              object
posting_date               object
promised_receipt_date      object
quantity_invoiced           int64
quantity                    int64
really_receipt_datetime    object
requested_receipt_date     object
dtype: object

In [19]:
lieferzeile.head(3)

,lieferzeile_no,order_no,buy_from_city,buy_from_post_code,buy_from_vendor_name,location_code,order_date,original_date,posting_date,promised_receipt_date,quantity_invoiced,quantity,really_receipt_datetime,requested_receipt_date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,22730,22730,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,15,15,01.01.1753 00:00:00,17.12.2019 00:00:00
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,2000,2000,01.01.1753 00:00:00,19.12.2019 00:00:00


In [20]:
lieferzeile.order_date = lieferzeile.order_date.map(lambda x: str(x)[:-9])
lieferzeile.original_date = lieferzeile.original_date.map(lambda x: str(x)[:-9])
lieferzeile.posting_date = lieferzeile.posting_date.map(lambda x: str(x)[:-9])
lieferzeile.promised_receipt_date = lieferzeile.promised_receipt_date.map(lambda x: str(x)[:-9])
lieferzeile.really_receipt_datetime = lieferzeile.really_receipt_datetime.map(lambda x: str(x)[:-9])
lieferzeile.requested_receipt_date = lieferzeile.requested_receipt_date.map(lambda x: str(x)[:-9])

In [21]:
lieferzeile.head(3)

,lieferzeile_no,order_no,buy_from_city,buy_from_post_code,buy_from_vendor_name,location_code,order_date,original_date,posting_date,promised_receipt_date,quantity_invoiced,quantity,really_receipt_datetime,requested_receipt_date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,23.09.2019,17.12.2019,30.12.2019,17.12.2019,22730,22730,16.12.2019,17.12.2019
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,15.11.2019,17.12.2019,30.12.2019,17.12.2019,15,15,01.01.1753,17.12.2019
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,19.12.2019,19.12.2019,30.12.2019,19.12.2019,2000,2000,01.01.1753,19.12.2019


In [22]:
# Dates in the really_receipt_datetime columns with a value of 1753-01-01 mean the field is blank or there's been an error in the input.
# we will filter out these rows (1,236 from 6012)
lieferzeile = lieferzeile[lieferzeile['really_receipt_datetime']!= '1753-01-01']
lieferzeile.shape

(6012, 14)

In [23]:
lieferzeile = lieferzeile.astype({'order_date': 'datetime64','original_date': 'datetime64','posting_date': 'datetime64',
'promised_receipt_date': 'datetime64','really_receipt_datetime': 'datetime64','requested_receipt_date': 'datetime64'})
lieferzeile.dtypes

lieferzeile_no                     object
order_no                           object
buy_from_city                      object
buy_from_post_code                 object
buy_from_vendor_name               object
location_code                      object
order_date                 datetime64[ns]
original_date              datetime64[ns]
posting_date               datetime64[ns]
promised_receipt_date      datetime64[ns]
quantity_invoiced                   int64
quantity                            int64
really_receipt_datetime    datetime64[ns]
requested_receipt_date     datetime64[ns]
dtype: object

In [24]:
lieferzeile.isna().sum()

lieferzeile_no             0
order_no                   0
buy_from_city              2
buy_from_post_code         4
buy_from_vendor_name       0
location_code              0
order_date                 0
original_date              0
posting_date               0
promised_receipt_date      0
quantity_invoiced          0
quantity                   0
really_receipt_datetime    0
requested_receipt_date     0
dtype: int64

In [25]:
# drop rows with nan values
lieferzeile = lieferzeile.dropna()
lieferzeile.shape

(6007, 14)

In [29]:
# drop duplicates
lieferzeile.drop_duplicates(keep='first')
lieferzeile.shape

(6007, 14)

In [26]:
lieferzeile.head(3)

,lieferzeile_no,order_no,buy_from_city,buy_from_post_code,buy_from_vendor_name,location_code,order_date,original_date,posting_date,promised_receipt_date,quantity_invoiced,quantity,really_receipt_datetime,requested_receipt_date
0,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,B-01,2019-09-23,2019-12-17,2019-12-30,2019-12-17,22730,22730,2019-12-16,2019-12-17
1,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,B-01,2019-11-15,2019-12-17,2019-12-30,2019-12-17,15,15,1753-01-01,2019-12-17
2,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,B-01,2019-12-19,2019-12-19,2019-12-30,2019-12-19,2000,2000,1753-01-01,2019-12-19


In [28]:
# export as csv file
lieferzeile.to_csv(r'lieferzeile_clean.csv', index=False);